# **Trabalho Final**
### Algoritmos e Programação com Linguagem Python
---

**Orientações Gerais:**

* Esta é o trabalho final prático onde serão trabalhados conceitos 
apresentados e exercitados em sala de aula.

* Aspéctos como clareza e organização do código, comentários, raciocínio lógico, coerência no nome de variáveis, utilização das estruturas mais adequadas para a solução dos problemas e o funcionamento adequado serão analisados e irão compor o valor da nota final.

* Procure entender exatamente o que se pede antes de iniciar a codificação. Quando se sabe o que é necessário fazer, a codificação fica mais fácil, mais eficiente e mais divertida ;-).

* Faça o upload do notebook completo nomeado da seguinte forma: *nome_sobrenome_RA.ipynb*. 

* No caso de grupos, informar ao professor os membros de cada grupo e cada um deverá importar o arquivo no Canvas individualmente.

* Arquivos com nomes incorretos serão desconsiderados!

* Não deixem dúvidas para trás! Acessem os grupos combinados ou entrem em contato com o professor. 

✌ Sucesso o/






# RMS Titanic

O RMS Titanic foi um navio de passageiros britânico operado pela White Star Line e construído pelos estaleiros da Harland and Wolff em Belfast. Foi a segunda embarcação da Classe Olympic de transatlânticos depois do RMS Olympic e seguido pelo HMHS Britannic. Projetado pelos engenheiros navais Alexander Carlisle e Thomas Andrews, sua construção começou em março de 1909 e ele foi lançado ao mar em maio de 1911. O Titanic foi pensado para ser o navio mais luxuoso e mais seguro de sua época, gerando lendas que era supostamente "inafundável".

A embarcação partiu em sua viagem inaugural de Southampton para Nova Iorque em 10 de abril de 1912, no caminho passando em Cherbourg-Octeville na França e por Queenstown na Irlanda. Ele colidiu com um iceberg às 23h40min do dia 14 de abril e afundou na madrugada do dia seguinte matando mais de 1500 pessoas, sendo um dos maiores desastres marítimos em tempos de paz de toda a história. Seu naufrágio destacou vários pontos fracos de seu projeto, deficiências nos procedimentos de evacuação de emergência e falhas nas regulamentações marítimas da época. Comissões de inquérito foram instauradas nos Estados Unidos e no Reino Unido, levando a mudanças nas leis internacionais de navegação que permanecem em vigor mais de um século depois.

Os destroços do Titanic foram procurados por décadas até serem encontrados em 1985 por uma equipe liderada por Robert Ballard. Ele se encontra a 3843 m de profundidade e a 650 km ao sudeste de Terra Nova no Canadá. Sua história e naufrágio permaneceram no imaginário popular durante décadas, levando a produção de vários livros e filmes a seu respeito, mais notavelmente o filme Titanic de 1997. Até hoje o Titanic permanece como um dos navios mais famosos da história, com seus destroços atraindo várias expedições de exploração ao longo dos anos.

*Fonte:* [WikiPedia](https://pt.wikipedia.org/wiki/RMS_Titanic)

--------

A equipe da [Udacity](https://udacity.com) disponibilizou informações sobre uma parte dos passageiros à bordo do RMS Titanic no dia daquela tragédia. Esse _dataset_ que contém informações reais vem sendo utilizado em diversos estudos de caso, além de servir de base para aprendizado de Ciência de Dados, Estatística e Machine Learning.

O arquivo **titanic_data.csv** contém informações reais sobre os passageiros organizadas da seguinte forma:

- Survived (Sobreviveu): 0 = Não, 1 = Sim
- Pclass (Classe): Classe de ingresso 1 = 1º, 2 = 2º, 3 = 3º
- Sex (Sexo): Sexo do passageiro
- Age (Idade): Idade em anos
- Sibsp: Quantidade de irmãos / cônjuges a bordo do Titanic
- Parch: Quantidade de pais / crianças a bordo do Titanic
- Ticket (Bilhete): Número do bilhete de embarque
- Fare (Tarifa): Tarifa paga pelo Passageiro
- Cabin (Cabine): Número de cabine
- Embarked (Embarque): Porto de Embarque (C = Cherbourg, Q = Queenstown, S = Southampton)

O objetivo deste estudo é obter algumas informações estatísticas sobre a tragédia e realizar alguns _insights_ a respeito dos tripulantes e das circunstâncias do acidente.

Efetue a carga do arquivo através da biblioteca _Pandas_ e a partir do _DataFrame_ gerado, efetue as operações necessárias para responder as questões.


In [1]:
import pandas as pd

class DataProcessor:

    def __init__(self):
      self.dataframes={}

    def init_df(self, name, df):
        self.dataframes[name] = {'original_df': df.copy(), 'states': [df.copy()], 'current_state': 0}
    
    def add_df_state(self, name, df):
        self.dataframes[name]['states'].append(df.copy())
        self.dataframes.get(name)['current_state'] += 1

    def get_current_state(self, name):
        return self.dataframes.get(name).get('current_state')
    
    def get_dataframe(self, name, original=True):
        if not original:
            return self.dataframes.get(name).get('original_df') 
        else:
            return self.dataframes.get(name).get('states')[self.get_current_state(name)]

    def undo(self, name):
        if self.get_current_state(name) == 0: 
          return
        self.dataframes.get(name)['states'] = self.dataframes.get(name).get('states')[:-1]
        self.dataframes.get(name)['current_state']-=1

    def remove_nan_values(self, name, columns=None):
        df = self.get_dataframe(name).copy()
        log = {}
        if not columns:
          columns = df.columns
        for column in columns:
          if df[column].isnull().values.any():
            log[column] = len(df[df[column].isnull()])
            if df.dtypes[column] == object:
              df.loc[df[column].isna(), column] = ''
            elif df.dtypes[column] == int or df.dtypes[column] == float:
              df.loc[df[column].isna(), column] = 0
        if log:
            self.add_df_state(name, df)
        return log
        
    def drop_columns(self, name, columns):
        df = self.get_dataframe(name).copy()
        try:
            df = df.drop(columns=columns)
            if len(df.columns) != len(self.get_dataframe(name).columns):
                self.add_df_state(name, df)
        except KeyError:
            raise("Column Invalid, please check columns name")

    def rename_columns(self, name, columns_dict):
        df = self.get_dataframe(name).copy()
        df = df.rename(columns=columns_dict)
        self.add_df_state(name, df)

    def replace_column_values(self, name, column, values_to_replace):
        df = self.get_dataframe(name).copy()
        if column not in df.columns:
            raise KeyError("Column Invalid")
        df = df.replace({column: values_to_replace})
        self.add_df_state(name, df)

    def read_file(self, path, name, sheet=None, encoding='UTF-8'):
      try:
        ext = path.rsplit(".")[1]
        if ext == 'xlsx':
          df = pd.read_excel(path)
        elif ext == 'csv':
          df = pd.read_csv(path)
        else:
          raise
        self.init_df(name, df)        

      except AttributeError:
        raise Exception("Invalid parameter")
      except:
        raise Exception("Error, invalid file path")

## Importando o DataSet

1 - Efetuar a leitura do dataset através do Pandas, gerando um DataFrame de nome tit, levando-se em consideração que o arquivo encontra-se na mesma pasta do notebook.

In [2]:
c = DataProcessor()
c.read_file('titanic_data.csv','tit')
tit = c.get_dataframe('tit', True)

## Conhecendo o DataSet

2 - Quantas linhas e colunas tem o dataset? Quais os tipos das colunas?

In [3]:
print("Linhas: ",tit.shape[0], "Colunas: ",tit.shape[1])
print("Tipos das colunas:\n"+ str(tit.dtypes))

Linhas:  891 Colunas:  12
Tipos das colunas:
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


3 - Mostre os dados dos dez primeiros e 8 últimos registros do dataset.

In [4]:
tit.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [5]:
tit.tail(8)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.500,NaN,S
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.050,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.125,NaN,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.450,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.000,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.750,NaN,Q


4 - Cálculos envolvendo colunas numéricas com dados faltantes podem sofrer impacto. É possível afirmar se há dados faltantes no dataset? Caso positivo, quais e quantos seriam esses dados? Preencha os dados faltantes de forma que não influenciem em operações futuras.

In [6]:
result = c.remove_nan_values('tit')
for key, value in result.items():
  print('Column:', key, 'Missing data:',value)

Column: Age Missing data: 177
Column: Cabin Missing data: 687
Column: Embarked Missing data: 2


## Adequando o DataSet

5 - Uma vez que algumas colunas não serão utilizadas, eventualmente é melhor excluí-las para que não interfiram na análise. Assim, exclua do dataset as colunas Sibsp, Parch e Ticket.

In [7]:
c.drop_columns('tit', ['SibSp','Parch','Ticket'])
c.get_dataframe('tit')

,PassengerId,Survived,Pclass,Name,Sex,Age,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,7.2500,,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,7.9250,,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,8.0500,,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,13.0000,,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,0.0,23.4500,,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,30.0000,C148,C


6 - Renomear as colunas restantes para a lingua portuguesa, utilizando os seguintes nomes de colunas: IdPassageiro, Sobreviveu, Classe, Nome, Sexo, Idade, Tarifa, Cabine e Embarque.

In [8]:
columns = {
    'PassengerId': 'IdPassageiro',
    'Survived': 'Sobreviveu',
    'Pclass': 'Classe',
    'Name': 'Nome',
    'Sex': 'Sexo',
    'Age': 'Idade',
    'Fare': 'Tarifa',
    'Embarked': 'Embarque',
    'Cabin': 'Cabine'
}
c.rename_columns('tit', columns)
c.get_dataframe('tit')

,IdPassageiro,Sobreviveu,Classe,Nome,Sexo,Idade,Tarifa,Cabine,Embarque
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,7.2500,,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,7.9250,,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,8.0500,,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,13.0000,,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,0.0,23.4500,,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,30.0000,C148,C


7 - Alterar o conteudo da coluna Sobreviveu para:
- 0 => Não
- 1 => Sim

In [9]:
replace_values = {0: 'Não', 1: 'Sim'}
c.replace_column_values('tit', 'Sobreviveu', replace_values)
c.get_dataframe('tit')

,IdPassageiro,Sobreviveu,Classe,Nome,Sexo,Idade,Tarifa,Cabine,Embarque
0,1,Não,3,"Braund, Mr. Owen Harris",male,22.0,7.2500,,S
1,2,Sim,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,71.2833,C85,C
2,3,Sim,3,"Heikkinen, Miss. Laina",female,26.0,7.9250,,S
3,4,Sim,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,53.1000,C123,S
4,5,Não,3,"Allen, Mr. William Henry",male,35.0,8.0500,,S
...,...,...,...,...,...,...,...,...,...
886,887,Não,2,"Montvila, Rev. Juozas",male,27.0,13.0000,,S
887,888,Sim,1,"Graham, Miss. Margaret Edith",female,19.0,30.0000,B42,S
888,889,Não,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,0.0,23.4500,,S
889,890,Sim,1,"Behr, Mr. Karl Howell",male,26.0,30.0000,C148,C


8 - Alterar o conteudo da coluna Sexo para:
- female => Mulher
- male   => Homem

In [10]:
replace_values = {'female': 'Mulher', 'male': 'Homem'}
c.replace_column_values('tit', 'Sexo', replace_values)
c.get_dataframe('tit')

,IdPassageiro,Sobreviveu,Classe,Nome,Sexo,Idade,Tarifa,Cabine,Embarque
0,1,Não,3,"Braund, Mr. Owen Harris",Homem,22.0,7.2500,,S
1,2,Sim,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Mulher,38.0,71.2833,C85,C
2,3,Sim,3,"Heikkinen, Miss. Laina",Mulher,26.0,7.9250,,S
3,4,Sim,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Mulher,35.0,53.1000,C123,S
4,5,Não,3,"Allen, Mr. William Henry",Homem,35.0,8.0500,,S
...,...,...,...,...,...,...,...,...,...
886,887,Não,2,"Montvila, Rev. Juozas",Homem,27.0,13.0000,,S
887,888,Sim,1,"Graham, Miss. Margaret Edith",Mulher,19.0,30.0000,B42,S
888,889,Não,3,"Johnston, Miss. Catherine Helen ""Carrie""",Mulher,0.0,23.4500,,S
889,890,Sim,1,"Behr, Mr. Karl Howell",Homem,26.0,30.0000,C148,C


## Alguns números

In [11]:
tit = c.get_dataframe('tit')

9 - Quantas mulheres e quantos homems estavam à bordo, de acordo com o dataset?

In [12]:
tit['Sexo'].value_counts()

Homem     577
Mulher    314
Name: Sexo, dtype: int64

10 - Quantos passageiros sobreviveram e quantos não sobreviveram?

In [13]:
tit['Sobreviveu'].value_counts()

Não    549
Sim    342
Name: Sobreviveu, dtype: int64

11 - Quantas mulheres não sobreviveram?

In [14]:
len(tit.loc[(tit['Sexo']=='Mulher') & (tit['Sobreviveu']=='Não')])

81

12 - Proporcionalmente, sobreviveram mais homens ou mais mulheres? Cite as proporções.

In [15]:
female_survived = len(tit.loc[(tit['Sexo']=='Mulher') & (tit['Sobreviveu']=='Sim')])
female_total = len(tit.loc[tit['Sexo']=='Mulher'])

male_survived = len(tit.loc[(tit['Sexo']=='Homem') & (tit['Sobreviveu']=='Sim')])
male_total = len(tit.loc[tit['Sexo']=='Homem'])

prop_female = round((female_survived/female_total)*100,2)
prop_male = round((male_survived/male_total)*100,2)

if prop_female > prop_male:
    print(f"Proporcionalmente sobreviveram mais mulheres!\n{prop_female}% de {female_total}\nEnquanto os Homens sobreviveram {prop_male}% de {male_total}")
else:
    print(f"Proporcionalmente sobreviveram mais homens!\n{prop_male}% de {male_total}\nEnquanto as Mulheres sobreviveram {prop_female}% de {female_total}")

Proporcionalmente sobreviveram mais mulheres!
74.2% de 314
Enquanto os Homens sobreviveram 18.89% de 577


13 - Levando-se em consideração a idade dos passageiros, qual a idade e quantidade de pessoas com o maior número de mortos?

In [16]:
mode = tit[(tit['Idade']!=0) & (tit['Sobreviveu']=='Não')]['Idade'].mode()
qty = tit[(tit['Idade']==mode[0]) & (tit['Sobreviveu']=='Não')].shape[0]
print(f"Idade: {mode[0]}\nQuantidade: {qty}")

Idade: 21.0
Quantidade: 19


14 - Qual a média de idade dos homens sobreviventes?

In [17]:
mean = tit[(tit['Idade']!=0) & (tit['Sobreviveu']=='Sim') & (tit['Sexo']=='Homem')]['Idade'].mean()
print("Média de idades dos homens sobreviventes: ", round(mean,2))

Média de idades dos homens sobreviventes:  27.28


15 - Levando-se em consideração passageiros prioritários (mulheres e crianças de até 15 anos independente do sexo) qual a proporção de sobreviventes por sexo?

In [18]:
filter = tit[(tit['Sobreviveu']=='Sim') & ((tit['Sexo']=='Mulher') | ((tit['Idade'] > 0) & (tit['Idade'] <= 15)))]['Sexo'].value_counts()
female_count = tit[tit['Sexo'] == 'Mulher']['Sexo'].value_counts()
male_count = tit[tit['Sexo'] == 'Homem']['Sexo'].value_counts()
prop_female = filter.Mulher/female_count
prop_male = filter.Homem/male_count

print(f'{float(round(prop_female*100,2))}% das mulheres sobreviveram! \n{float(round(prop_male*100,2))}% dos homens sobreviveram!')

74.2% das mulheres sobreviveram! 
3.64% dos homens sobreviveram!


16 - Qual a quantidade de passageiros por classe?

In [19]:
tit['Classe'].value_counts()

3    491
1    216
2    184
Name: Classe, dtype: int64

17 - Qual o percentual de sobreviventes por classe?

In [20]:
total = tit[(tit['Sobreviveu']=='Sim')]['Classe'].value_counts()
print(f"Classe {total.index[0]}: {round((total.values[0]/total.values.sum())*100,2)}%")
print(f"Classe {total.index[1]}: {round((total.values[1]/total.values.sum())*100,2)}%")
print(f"Classe {total.index[2]}: {round((total.values[2]/total.values.sum())*100,2)}%")

Classe 1: 39.77%
Classe 3: 34.8%
Classe 2: 25.44%


18 - Crie um dataframe que demonstre a quantidade de sobreviventes e não sobreviventes, agrupados por sexo e classe.

In [21]:
new_df = pd.DataFrame(tit.groupby(['Classe','Sobreviveu'])['Sexo'].value_counts())
new_df

Sexo
Classe Sobreviveu Sexo        
1      Não        Homem     77
                  Mulher     3
       Sim        Mulher    91
                  Homem     45
2      Não        Homem     91
                  Mulher     6
       Sim        Mulher    70
                  Homem     17
3      Não        Homem    300
                  Mulher    72
       Sim        Mulher    72
                  Homem     47

19 - Dos homens com idade entre 24 e 30 anos quantos da classe 3 sobreviveram? Quantos da classe 2 não sobreviveram?

In [22]:
print("Homens sobreviventes de 24 a 30 anos de classe 3: ",tit[((30 <= tit['Idade']) & (tit['Idade'] >= 24) & (tit['Sexo'] == 'Homem') & (tit['Classe'] == 3) & (tit['Sobreviveu'] == 'Sim'))]['Classe'].value_counts().values[0])
print("Homens mortos de 24 a 30 anos de classe 2: ",tit[((30 <= tit['Idade']) & (tit['Idade'] >= 24) & (tit['Sexo'] == 'Homem') & (tit['Classe'] == 2) & (tit['Sobreviveu'] == 'Sim'))]['Classe'].value_counts().values[0])

Homens sobreviventes de 24 a 30 anos de classe 3:  10
Homens mortos de 24 a 30 anos de classe 2:  5


20 - Calcule a probabilidade condicional de uma pessoa sobreviver, dado seu sexo e a classe em que estava viajando:
- P(S= true | G=female,C=1)
- P(S= true | G=female,C=2)
- P(S= true | G=female,C=3)
- P(S= true | G=male,C=1)
- P(S= true | G=male,C=2)
- P(S= true | G=male,C=3)

In [23]:
Mulher_Classe_Um = tit[(tit['Sexo']=='Mulher') & (tit['Classe']==1)]['Sobreviveu'].value_counts()
Mulher_Classe_Dois = tit[(tit['Sexo']=='Mulher') & (tit['Classe']==2)]['Sobreviveu'].value_counts()
Mulher_Classe_Tres = tit[(tit['Sexo']=='Mulher') & (tit['Classe']==3)]['Sobreviveu'].value_counts()
Homem_Classe_Um = tit[(tit['Sexo']=='Homem') & (tit['Classe']==1)]['Sobreviveu'].value_counts()
Homem_Classe_Dois = tit[(tit['Sexo']=='Homem') & (tit['Classe']==2)]['Sobreviveu'].value_counts()
Homem_Classe_Tres = tit[(tit['Sexo']=='Homem') & (tit['Classe']==3)]['Sobreviveu'].value_counts()
print(f'P(S= true | G=female,C=1): {round((Mulher_Classe_Um.Sim /Mulher_Classe_Um.values.sum()*100),2)}%')
print(f'P(S= true | G=female,C=2): {round((Mulher_Classe_Dois.Sim /Mulher_Classe_Dois.values.sum()*100),2)}%')
print(f'P(S= true | G=female,C=3): {round((Mulher_Classe_Tres.Sim /Mulher_Classe_Tres.values.sum()*100),2)}%')
print(f'P(S= true | G=male,C=1): {round((Homem_Classe_Um.Sim /Homem_Classe_Um.values.sum()*100),2)}%')
print(f'P(S= true | G=male,C=2): {round((Homem_Classe_Dois.Sim /Homem_Classe_Dois.values.sum()*100),2)}%')
print(f'P(S= true | G=male,C=3): {round((Homem_Classe_Tres.Sim /Homem_Classe_Tres.values.sum()*100),2)}%')

P(S= true | G=female,C=1): 96.81%
P(S= true | G=female,C=2): 92.11%
P(S= true | G=female,C=3): 50.0%
P(S= true | G=male,C=1): 36.89%
P(S= true | G=male,C=2): 15.74%
P(S= true | G=male,C=3): 13.54%
